In [1]:
import math
import torch
import random
import torch.nn as nn
import torch.optim as optim
import numpy as np
import gymnasium as gym
import os

from torch.utils.tensorboard import SummaryWriter
from torch.distributions.normal import Normal

In [2]:
# Hyperparameters
CONFIG_NUMBER = 14

OBSERVATION_SIZE = 64
NB_FRAMES = 1
NB_ENVS = 1
CUDA = False
SEED = 2307

MAX_EPISODE_LENGTH = 12000
LEARNING_RATE = 1e-4
NB_STEPS = 2048
TOTAL_TIMESTEPS = 1.5e6
RUN_NAME = "A2C with GAE and one env"
LR_SCHEDULING = "Linear"
GAMMA = 0.99
VF_COEF = 0.5
ENT_COEF = 0.01
MAX_GRAD_NORM = 0.5
GAE_LAMBDA = 0.95

In [3]:
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

def make_env():
    def thunk():
        env = gym.make("CarRacing-v3", render_mode="rgb_array", lap_complete_percent=0.95, domain_randomize=False, continuous=True, max_episode_steps=MAX_EPISODE_LENGTH)
        env = gym.wrappers.RecordEpisodeStatistics(env)
        env = gym.wrappers.ResizeObservation(env, (OBSERVATION_SIZE, OBSERVATION_SIZE))
        env = gym.wrappers.GrayscaleObservation(env)
        env = gym.wrappers.FrameStackObservation(env, NB_FRAMES)
        return env

    return thunk

envs = gym.vector.SyncVectorEnv([make_env() for _ in range(NB_ENVS)])

In [4]:
# Taken from TP5

def layer_init(layer, std=np.sqrt(2), bias_const=0.0):
    torch.nn.init.orthogonal_(layer.weight, std)
    torch.nn.init.constant_(layer.bias, bias_const)
    return layer

class Agent(nn.Module):
    def __init__(self, envs, nb_frames, image_size):
        super(Agent, self).__init__()
        self.image_size = image_size
        self.nb_frames = nb_frames

        # Actor network
        self.actor_network = self.build_network()
        self.actor_mean = layer_init(
            nn.Linear(512, np.prod(envs.single_action_space.shape)),
            std=0.01
        )
        self.actor_logstd = nn.Parameter(torch.zeros(1, np.prod(envs.single_action_space.shape)))

        # Critic network
        self.critic_network = self.build_network()
        self.critic = layer_init(nn.Linear(512, 1), std=1)

    def build_network(self):
        stride = [4, 2, 1]
        kernel_size = [8, 4, 3]
        input_channels = [self.nb_frames, 32, 64]
        output_channels = [32, 64, 64]
        image_size = self.image_size

        layers = []
        for i in range(len(stride)):
            layers.append(layer_init(nn.Conv2d(input_channels[i], output_channels[i], kernel_size[i], stride=stride[i])))
            layers.append(nn.ReLU())
            image_size = math.floor(((image_size - kernel_size[i]) / stride[i]) + 1)

        layers.append(nn.Flatten())
        layers.append(layer_init(nn.Linear(output_channels[-1] * image_size * image_size, 512)))
        layers.append(nn.ReLU())
        
        return nn.Sequential(*layers)

    def get_value(self, x):
        hidden = self.critic_network(x / 255.0)
        return self.critic(hidden)

    def get_action_and_value(self, x, action=None):
        actor_hidden = self.actor_network(x / 255.0)
        action_mean = self.actor_mean(actor_hidden)
        action_logstd = self.actor_logstd.expand_as(action_mean)
        action_std = torch.exp(action_logstd)
        probs = Normal(action_mean, action_std)
        if action is None:
            action = probs.sample()
        
        critic_hidden = self.critic_network(x / 255.0)
        value = self.critic(critic_hidden)

        return action, probs.log_prob(action).sum(1), probs.entropy().sum(1), value

In [5]:
device = torch.device("cuda" if torch.cuda.is_available() and CUDA else "cpu")
agent = Agent(envs, NB_FRAMES, OBSERVATION_SIZE).to(device)
optimizer = optim.Adam(agent.parameters(), lr=LEARNING_RATE, eps=1e-5)

obs = torch.zeros((NB_STEPS, NB_ENVS) + envs.single_observation_space.shape).to(device)
actions = torch.zeros((NB_STEPS, NB_ENVS) + envs.single_action_space.shape).to(device)
rewards = torch.zeros((NB_STEPS, NB_ENVS)).to(device)
values = torch.zeros((NB_STEPS, NB_ENVS)).to(device)
current_logprobs = torch.zeros((NB_STEPS, NB_ENVS)).to(device)
previous_logprobs = torch.zeros((NB_STEPS, NB_ENVS)).to(device)

global_step = 0
next_obs, _ = envs.reset(seed=SEED)
next_obs = torch.Tensor(next_obs).to(device)
next_done = torch.zeros(NB_ENVS).to(device)

if RUN_NAME is not None:
    import wandb

    config = {
        "OBSERVATION_SIZE": OBSERVATION_SIZE,
        "NB_FRAMES": NB_FRAMES,
        "NB_ENVS": NB_ENVS,
        "SEED": SEED,
        "MAX_EPISODE_LENGTH": MAX_EPISODE_LENGTH,
        "LEARNING_RATE": LEARNING_RATE,
        "NB_STEPS": NB_STEPS,
        "TOTAL_TIMESTEPS": TOTAL_TIMESTEPS,
        "LR_SCHEDULING": LR_SCHEDULING,
        "GAMMA": GAMMA,
        "VF_COEF": VF_COEF,
        "CUDA": CUDA,
        "ENT_COEF": ENT_COEF,
        "CONFIG_NUMBER": CONFIG_NUMBER,
        "GAE_LAMBDA": GAE_LAMBDA
    }

    wandb.init(
        entity="Rl2025-project",
        project="RL Project",
        name=RUN_NAME,
        config=config,
        sync_tensorboard=True,  # auto-upload sb3's tensorboard metrics
        monitor_gym=True,       # auto-upload des vidéos de l'agent
        # save_code=True,       # optionnel
    )
    
    writer = SummaryWriter(f"runs/{RUN_NAME}")

/home/florenthervers/miniconda3/envs/car_racing_env/lib/python3.10/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: flo230702 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
while global_step < TOTAL_TIMESTEPS:
    # Annealing the rate if instructed to do so.
    if LR_SCHEDULING == "Linear":
        frac = 1.0 - global_step / TOTAL_TIMESTEPS
        lrnow = frac * LEARNING_RATE
        optimizer.param_groups[0]["lr"] = lrnow

    for step in range(0, NB_STEPS):
        global_step += NB_ENVS
        obs[step] = next_obs

        with torch.no_grad():
            action, logprob, _, value = agent.get_action_and_value(next_obs)
            values[step] = value.flatten()
            current_logprobs[step] = logprob

        actions[step] = action

        next_obs, reward, terminations, truncations, infos = envs.step(action.cpu().numpy())
        next_done = np.logical_or(terminations, truncations)
        rewards[step] = torch.tensor(reward).to(device).view(-1)
        next_obs, next_done = torch.Tensor(next_obs).to(device), torch.Tensor(next_done).to(device)
        
        if "episode" in infos:
            completed_episodes = infos["_episode"]
            episodic_returns = infos["episode"]["r"][completed_episodes]
            episodic_lengths = infos["episode"]["l"][completed_episodes]

            for episodic_return, episodic_length in zip(episodic_returns, episodic_lengths):
                print(f"global_step={global_step}, episodic_return={episodic_return}")
                if RUN_NAME != None:
                    writer.add_scalar("charts/episodic_return", episodic_return, global_step)
                    writer.add_scalar("charts/episodic_length", episodic_length, global_step)
        
        # Break when one of the environement as reached a terminal state
        if torch.any(next_done):
            break
    
    if GAE_LAMBDA == None:
        R = torch.Tensor([0 if next_done[i] == True else values[-1][i] for i in range(len(next_done))]).to(device)
        returns = torch.zeros_like(rewards)
        advantages = torch.zeros_like(rewards)

        for i in reversed(range(step)):
            R = rewards[i] + GAMMA * R
            returns[i] = R
            advantages[i] = returns[i] - values[i]
    
    else:
        next_value = agent.get_value(next_obs).reshape(1, -1)
        advantages = torch.zeros_like(rewards)
        lastgaelam = 0

        for t in reversed(range(step)):
            if t == step - 1:
                nextnonterminal = 1.0 - next_done
                nextvalue = next_value
            else:
                nextnonterminal = 1.0 - torch.zeros_like(next_done)
                nextvalue = values[t + 1]
            
            delta = rewards[t] + GAMMA * nextvalue * nextnonterminal - values[t]
            advantages[t] = lastgaelam = delta + GAMMA * GAE_LAMBDA * nextnonterminal * lastgaelam

        returns = advantages + values

    # Normalize the advantages
    advantages = (advantages - advantages.mean()) / (advantages.std() + 1e-8)

    value_loss = torch.zeros((step, NB_ENVS))
    actor_loss = torch.zeros((step, NB_ENVS))
    entropy_term = torch.zeros((step, NB_ENVS))

    for i in range(step):
        _, logprob, ent, value = agent.get_action_and_value(obs[i], actions[i])
        value = value.flatten()

        actor_loss[i] = -logprob * advantages[i]
        value_loss[i] = (returns[i] - value)**2
        entropy_term[i] = ent

    actor_loss = actor_loss.mean()
    value_loss = 0.5 * value_loss.mean()
    entropy_term = entropy_term.mean()

    loss = actor_loss + VF_COEF * value_loss - ENT_COEF * entropy_term
    approx_kl = (previous_logprobs - current_logprobs).mean() if global_step > NB_ENVS * NB_STEPS else 0.0

    previous_logprobs = current_logprobs.detach().clone()
    
    optimizer.zero_grad()
    loss.backward()
    nn.utils.clip_grad_norm_(agent.parameters(), MAX_GRAD_NORM)
    optimizer.step()

    # logging for the losses + learning rate
    if RUN_NAME != None:
        writer.add_scalar("losses/total_loss", loss.item(), global_step)
        writer.add_scalar("losses/actor_loss", actor_loss.mean().item(), global_step)
        writer.add_scalar("losses/value_loss", value_loss.mean().item(), global_step)
        writer.add_scalar("losses/entropy", entropy_term.mean().item(), global_step)
        writer.add_scalar("losses/approx_kl", approx_kl, global_step)
        writer.add_scalar("charts/learning_rate", lrnow, global_step)
    
    
# Save the model at the end of training
save_path = f"trained_models/a2c/a2c_config{CONFIG_NUMBER}.pt"
os.makedirs("trained_models/a2c", exist_ok=True)
torch.save({
    "model_state_dict": agent.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "config": {
        "OBSERVATION_SIZE": OBSERVATION_SIZE,
        "NB_FRAMES": NB_FRAMES,
        "NB_ENVS": NB_ENVS,
        "SEED": SEED,
        "MAX_EPISODE_LENGTH": MAX_EPISODE_LENGTH,
        "LEARNING_RATE": LEARNING_RATE,
        "NB_STEPS": NB_STEPS,
        "TOTAL_TIMESTEPS": TOTAL_TIMESTEPS,
        "LR_SCHEDULING": LR_SCHEDULING,
        "GAMMA": GAMMA,
        "VF_COEF": VF_COEF,
        "GAE_LAMBDA": GAE_LAMBDA,
    }
}, save_path)
print(f"Model saved to {save_path}")
if RUN_NAME != None:
    wandb.finish()

global_step=12000, episodic_return=-1137.6770538244225
global_step=22378, episodic_return=-1067.5363057326347
global_step=28341, episodic_return=-632.942105263211
global_step=40342, episodic_return=-1089.2508143323512
global_step=52343, episodic_return=-1133.7979094077332
global_step=55070, episodic_return=-334.82876712327857
global_step=57902, episodic_return=-337.39739413679854
global_step=69903, episodic_return=-1146.843853820647
global_step=73913, episodic_return=-413.55167785235085
global_step=79398, episodic_return=-600.3295202952484
global_step=90367, episodic_return=-992.4317073172334
global_step=102368, episodic_return=-962.3762376239507
global_step=114369, episodic_return=-1141.7177914111037
global_step=126370, episodic_return=-1179.0209790210024
global_step=138371, episodic_return=-1181.0606060606276
global_step=150372, episodic_return=-1183.9228295820124
global_step=162373, episodic_return=-1182.5174825175027
global_step=174374, episodic_return=-1184.3750000000184
global_st

charts/episodic_length,▃█▁▂▇███████████████████████████████████
charts/episodic_return,▂▂█▃▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
charts/learning_rate,██▇▇▇▇▇▇▇▆▆▆▆▆▆▅▅▄▄▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁
global_step,▁▁▁▁▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇███
losses/actor_loss,▅▁▄▄▄▅▃▄▄▆▆▅▂█▄▇▂▆▇█▅▂▁▆▆▄▇▆▂▂▄▃▅▃▃▄▅█▄▃
losses/approx_kl,▅▆▆▄▅▃▇▆█▇▅▇▄▃▇▁▇▇▃▄▄▅▇▅▅▆█▇▄▂▇▅█▇▆▇▅█▂▄
losses/entropy,▁▁▁▁▁▂▂▂▂▂▂▃▄▄▄▅▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇█████████
losses/total_loss,▇█▁▆████████████████████████████████████
losses/value_loss,█▄▁▂▄▂▁▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▂▁▁▁▁▁▂▂▁▁▁▂▂▁▂
charts/episodic_length,12000
charts/episodic_return,-1182.5784
